In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from torch.utils.data import TensorDataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


In [3]:
## Sklearn Stuff
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Dataset preparation

In [4]:
train_instances = pd.read_csv("../data_forecasting/Train.csv")
train_instances.tail()

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered
35748,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27133,0,50,25,0,25,8,0,25.0
35749,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27138,150,0,0,0,150,0,0,0.0
35750,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27132,1,10,4,0,7,13,0,0.0
35751,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27134,8,0,8,0,0,10,0,0.0
35752,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS17005,48,0,2,0,46,1,0,0.0


In [ ]:
train_instances['timestamp'] = train_instances['year'].astype('str') + "-" + train_instances['month'].astype('str')
train_instances.drop(columns={'region', 
                      'district', 
                      'stock_initial', 
                      'stock_received', 
                      'stock_adjustment',
                      'stock_end',
                      'average_monthly_consumption',
                      'stock_stockout_days',
                      'stock_ordered'},
                      inplace=True)

In [ ]:
train_instances['timestamp']=pd.to_datetime(train_instances['timestamp'])
train_instances.index= train_instances.timestamp
train_instances.drop(columns={'timestamp'},inplace=True)

## Sort Dataframe by Date - Causality in Preds
train_instances.sort_values(by='timestamp', inplace=True)
train_instances.tail()

In [ ]:
site_encoder = LabelEncoder()
product_encoder = LabelEncoder()
#train_instances.drop(columns=['timestamp'], inplace=True)
train_instances['site_code_encoded'] = site_encoder.fit_transform(train_instances['site_code'])
train_instances['product_code_encoded'] = product_encoder.fit_transform(train_instances['product_code'])
train_instances.tail()

In [ ]:
## feature definition
features = ['year','month','site_code_encoded','product_code_encoded']
label = ['stock_distributed']

In [ ]:
final_dataset=train_instances[features+label]
final_dataset.tail()

In [ ]:
## Train/Test Split
train_size = 0.8
train_df = final_dataset[:int(train_size*(len(final_dataset)))]
valid_df = final_dataset[int(train_size*(len(final_dataset))):]
valid_df

In [ ]:
## Min/Max Scaler
feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

x_train = feature_scaler.fit_transform(train_df[features])
x_valid = feature_scaler.transform(valid_df[features])

y_train = label_scaler.fit_transform(train_df[label])
y_valid = label_scaler.transform(valid_df[label])
y_train

In [ ]:
## Pytorch Data loaders
batch_size = 16

train_features = torch.Tensor(x_train)
train_targets = torch.Tensor(y_train)
val_features = torch.Tensor(x_valid)
val_targets = torch.Tensor(y_valid)

train_dataset = TensorDataset(train_features, train_targets)
val_dataset = TensorDataset(val_features, val_targets)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

## PyTorch Modeling

In [ ]:
class LSTMPredictor(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers, dropout):
        super(LSTMPredictor, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers,
          dropout=dropout
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)
        self.reset_hidden_state()
        
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
        )
    def forward(self, sequences):
        lstm_out, self.hidden = self.lstm(
          sequences.view(len(sequences), self.seq_len, -1),
          self.hidden
        )
        last_time_step = \
          lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred

In [ ]:
model_params = {
    'n_features': x_train.shape[1],
    'n_hidden': 64,
    'seq_len': 1,
    'n_layers': 2,
    'dropout': 0.1
}

train_params = {
    'learning_rate': 1e-5,
    'weight_decay': 1e-6,
    'n_epochs': 10
    
}

In [ ]:
def train(train_data, valid_data, model_params, train_params):
    ## Train Definitions
    N_EPOCHS = train_params['n_epochs']
    model = LSTMPredictor(**model_params)
    loss_fn = nn.MSELoss(reduction="mean")
    optimizer = optim.Adam(model.parameters(), 
                           lr=train_params['learning_rate'], 
                           weight_decay=train_params['weight_decay'])
    ## Start training
    for epoch in range(N_EPOCHS):
        print(f'Epoch {epoch}')
        for step, (x,y) in enumerate(train_data):
            model.train()
            optimizer.zero_grad()
            model.reset_hidden_state()
            output = model(x)
            loss = loss_fn(output, y)
            loss.backward()
            optimizer.step()
            if step%500 == 0:
                for step_valid, (x_valid,y_valid) in enumerate(valid_data):
                    #model.eval()
                    with torch.no_grad():
                        output_valid = model.forward(x_valid)
                    mse = mean_squared_error(y_valid.detach().numpy(), output_valid.detach().numpy())
                print(f"Iteration: {step}; Loss: {loss.item()}; MSE: {mse}")
    return model.eval()

In [ ]:
model_lstm = train(train_loader, val_loader, model_params, train_params)

## Checking some preds

In [ ]:
train_instances.stock_distributed.min(), train_instances.stock_distributed.max()

In [ ]:
train_instances[train_instances.stock_distributed==1728]

In [ ]:
train_instances.tail(10)

In [ ]:
def get_1step_preds(year, month, site_code, product_code):
    site_encoded = site_encoder.transform([site_code])[0]
    product_encoded = product_encoder.transform([product_code])[0]
    x_vector = feature_scaler.transform([[year, month, site_encoded, product_encoded]])
    print(x_vector)
    model_lstm.eval()
    pred = model_lstm(torch.tensor(x_vector.astype(np.float32)))
    print(pred)
    return label_scaler.inverse_transform(pred.detach().numpy()).item()

In [ ]:
label_scaler.inverse_transform([[0.1]])

In [ ]:
get_1step_preds(2019, 6, 'C1007', 'AS27137')

In [ ]:
get_1step_preds(2019, 6, 'C5001', 'AS27137')

In [ ]:
get_1step_preds(2019, 6, 'C1007', 'AS27139')

## Sources
- https://colab.research.google.com/drive/1enI68fTdPI2w5KKv6jyL0Lcq9Zg3BbLx?usp=sharing#scrollTo=A5guw4joPA8q
- https://curiousily.com/posts/time-series-forecasting-with-lstm-for-daily-coronavirus-cases/